In [ ]:
import pandas as pd
from google.colab import files
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
torch.manual_seed(42)

In [ ]:
tqdm.pandas()

# Data

In [ ]:
train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_dataset.csv')
val = pd.read_csv('val_dataset.csv')

In [ ]:
# создаём датасет с примерами для few-shot
df_examples = pd.DataFrame()

for level in sorted(train['cefr_level'].unique()):
    df_examples = pd.concat([df_examples, train[train['cefr_level'] == level].sample(2, random_state=42)])

In [ ]:
# сбалансированные примеры (по два на уровень)
df_examples

,text,cefr_level
2179,Весь день будильник звонил неожиданно.,A1
18484,"Пятигорск славился своим чистым воздухом, мине...",A1
2013,Рыба начала двигаться и тянуть лодку за собой.,A2
14980,"Иностранцы всегда знали, что русские любят ско...",A2
1865,"«Всё изме нилось, — подумал Степан Аркадьич, —...",B1
15180,Ты вышел из детской и сказал мне: — Дядечка!,B1
1995,В этой аттестации как бы разработана психологи...,B2
15111,"— О боже, какое несчастие!",B2
1822,"Савка пел басом, откашливаясь в сторону и прик...",C1
15054,Леди-Яблоко была одна.,C1


In [ ]:
# конвертируем в строку, разделяем текст и уровень
examples = ''
for _, row in df_examples.iterrows():
    example = f"ТЕКСТ: {row['text']}\nУРОВЕНЬ: {row['cefr_level']}\n\n"
    examples += example

In [ ]:
# пропмпт
# указываем, в каком формате ожидаем ответ
# добавляем примеры few-shot
prompt = """
Определи уровень русскоязычного текста по шкале CEFR.
Уровень данного текста может быть A1, A2, B1, B2, C1, или C2.
В ответе укажи только уровень CEFR в формате A1, A2, B1, B2, C1 или C2, ничего больше не пиши.

Примеры текстов и их уровни: {examples}

Для следующего текста определи уровень сложности CEFR.

ТЕКСТ: {text}
УРОВЕНЬ:
"""

#  LLM T-lite

In [ ]:
# используем модель T-lite
model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto" if device.type == "cuda" else None
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
def get_level_llm(text, prompt, examples):
    '''
    Функция, которая по заданному пропмпту, примерам и тексту определяет уровень текста.
    Обращается к модели T-lite и выводит её ответ.
    '''
    messages = [
        {"role": "system", "content": "Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - быть полезным диалоговым ассистентом."},
        {"role": "user", "content": prompt.format(examples=examples,
                                                  text=text)}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=10
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [ ]:
# разбиваем тестовый датасет на две части, чтобы сессия не прервалась
test_first_half = test.iloc[:len(test)//2]

In [ ]:
test_first_half['t-lite_cefr'] = (
    test_first_half['text']
    .progress_apply(
        get_level_llm,
        args=(prompt,
              examples)
    )
)

100%|██████████| 1322/1322 [1:59:06<00:00,  5.41s/it]
/tmp/ipython-input-1750264083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_first_half['t-lite_cefr'] = (


In [ ]:
# сохраняем файл
test_first_half.to_csv('test_first_half.csv')
files.download('test_first_half.csv')

In [ ]:
# то же самое со второй половиной датасета
test_second_half = test.iloc[len(test)//2:]

In [ ]:
test_second_half['t-lite_cefr'] = (
    test_second_half['text']
    .progress_apply(
        get_level_llm,
        args=(prompt,
              examples)
    )
)

100%|██████████| 1323/1323 [1:59:03<00:00,  5.40s/it]
/tmp/ipython-input-2723577299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_second_half['t-lite_cefr'] = (


In [ ]:
test_second_half.to_csv('test_second_half.csv')
files.download('test_second_half.csv')

In [ ]:
# объединяем половинки датасетов и считаем метрики
test_whole = pd.concat([test_first_half, test_second_half])

In [ ]:
print(f"Accuracy: {accuracy_score(test_whole['cefr_level'], test_whole['t-lite_cefr'])}")

Accuracy: 0.21247637051039697


In [ ]:
print(f"F1-macro: {f1_score(test_whole['cefr_level'], test_whole['t-lite_cefr'], average='macro')}")

F1-macro: 0.16007365883986469


In [ ]:
test_whole['t-lite_cefr'].value_counts()

,count
t-lite_cefr,
B1,1318
B2,612
A2,584
A1,127
C2,3
C1,1


Как можно заметить, низкое качество модели обосновывается тем, что она склонна присваивать текстам средние уровни: B1 или B2, а также A2. При этом имеется перекос в сторону более простых текстов. Возможно, её смущает длина текста: так как это просто предложения, она считает, что они не могут быть уровня C1 или C2.

In [ ]:
# сохраняем итоговый файл
test_whole.to_csv('test_whole.csv')
files.download('test_whole.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>